In [1]:
%%configure -f
{
  "conf": {
    "spark.executor.instances": "4",
    "spark.executor.cores": "1",
    "spark.executor.memory": "2g",
    "spark.driver.memory": "2g"
  }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
393,application_1764662801237_0395,pyspark,idle,Link,Link,None,
405,application_1764662801237_0407,pyspark,idle,Link,Link,None,
412,application_1764662801237_0414,pyspark,idle,Link,Link,None,
416,application_1764662801237_0418,pyspark,idle,Link,Link,None,


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, trim, count
import time

start_time = time.time()

spark = SparkSession.builder.getOrCreate()

# Enable cross join for nested loop
spark.conf.set("spark.sql.crossJoin.enabled", "true")



crime_2010_2019 = spark.read.option("header", "true").csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv"
)
crime_2020_2025 = spark.read.option("header", "true").csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv"
)
combined_crime = crime_2010_2019.unionByName(crime_2020_2025)

mo_df = spark.read.text(
    "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/MO_codes.txt"
).withColumn("split_cols", split(col("value"), " ", 2)) \
 .withColumn("MO_Code", col("split_cols").getItem(0)) \
 .withColumn("MO_Desc", col("split_cols").getItem(1)) \
 .drop("value", "split_cols")

combined_crime_exploded = combined_crime.withColumn(
    "MO_Code",
    explode(split(col("Mocodes"), " "))
).withColumn("MO_Code", trim(col("MO_Code")))

# Shuffle Replicate Nested Loop Join
joined_df_nested_loop = combined_crime_exploded.crossJoin(mo_df)\
    .filter(trim(combined_crime_exploded.MO_Code) == trim(mo_df.MO_Code))

mo_summary_nested_loop = (
    joined_df_nested_loop
    .groupBy("MO_Code", "MO_Desc")
    .agg(count("*").alias("Frequency"))
    .filter(
        (col("MO_Code").isNotNull()) & (trim(col("MO_Code")) != "") &
        (col("MO_Desc").isNotNull()) & (trim(col("MO_Desc")) != "")
    )
    .orderBy(col("Frequency").desc())
)

results_nested_loop = mo_summary_nested_loop.collect()



page_size = 50
total_rows = len(results_nested_loop)

print(f"{'MO Code':<10} | {'Description':<50} | {'Frequency':<10}")
print("-" * 80)

for i in range(0, total_rows, page_size):
    chunk = results_nested_loop[i:i + page_size]
    for row in chunk:
        print(f"{row['MO_Code']:<10} | {row['MO_Desc']:<50} | {row['Frequency']:<10}")

end_time = time.time()

print(f"\nShuffle Replicate Nested Loop Join Execution Time: {end_time - start_time:.2f} sec\n")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
455,application_1764662801237_0457,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
[AMBIGUOUS_REFERENCE] Reference `MO_Code` is ambiguous, could be: [`MO_Code`, `MO_Code`].
Traceback (most recent call last):
  File "/mnt1/yarn/usercache/livy/appcache/application_1764662801237_0457/container_1764662801237_0457_01_000001/pyspark.zip/pyspark/sql/group.py", line 186, in agg
    jdf = self._jgd.agg(exprs[0]._jc, _to_seq(self.session._sc, [c._jc for c in exprs[1:]]))
  File "/mnt1/yarn/usercache/livy/appcache/application_1764662801237_0457/container_1764662801237_0457_01_000001/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/mnt1/yarn/usercache/livy/appcache/application_1764662801237_0457/container_1764662801237_0457_01_000001/pyspark.zip/pyspark/errors/exceptions/captured.py", line 185, in deco
    raise converted from None
pyspark.errors.exceptions.captured.AnalysisException: [AMBIGUOUS_REFERENCE] Reference `MO_Code` is ambiguous, could be: [`MO_Code`, `MO_Code`].

